### Load Libraries  

In [ ]:
!pip install python-terrier

In [ ]:
import pyterrier as pt
#if not pt.started():
 #   pt.init()

In [ ]:
pt.version()

In [ ]:
#install the Arabic stop words library
!pip install Arabic-Stopwords

In [ ]:
#we need to import the following libraries.
import pandas as pd
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)
import numpy as np
import re
from snowballstemmer import stemmer
from tqdm import tqdm
import arabicstopwords.arabicstopwords as stp

### Data Cleaning

In [ ]:
######################### removing Stop Words function ####################################

def remove_stop_words(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

############################# normalize function ###########################################

def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

############################# stemming function ############################################

ar_stemmer = stemmer("arabic")
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])


############################# perform first group of preprocessing steps ###############################


def preprocess1(sentence):
  sentence =remove_stop_words(sentence)
  sentence =normalize(sentence)
  return sentence


############################# normalize_tweets function ############################################


def normalize_tweets(text):
   if text is np.nan:
    return ""
   else:
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"@[\w]*", " ", text) # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    accents = re.compile(r'[\u064b-\u0652\u0640]') # harakaat and tatweel (kashida) to remove

    arabic_punc= re.compile(r'[\u0621-\u063A\u0641-\u064A\d+]+') # Keep only Arabic letters/do not remove numbers
    text=' '.join(arabic_punc.findall(accents.sub('',text)))
    text = text.strip()
    return text

############################# perform Second group of preprocessing steps ###############################

def preprocess2(sentence):
  sentence =remove_stop_words(sentence)
  sentence =normalize_tweets(sentence)
  return sentence

### Load Dataset

In [ ]:

dataset_links=["https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-01.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-02.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-03.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-04.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-05.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-06.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-07.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-08.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-09.txt",
              "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-10.txt"]

full_data=pd.DataFrame()
for i in tqdm(range(len(dataset_links))):
    tweets=pd.read_csv(dataset_links[i], sep='\t')
    full_data=pd.concat([full_data,tweets],ignore_index=True)
full_data.reset_index(inplace=True,drop=True)

#the docno will be our tweetID
full_data["docno"]=full_data["tweetID"].astype(str)


100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


In [ ]:
# preparing the data for index
full_data['text']=full_data['tweetText'].apply(preprocess2)

full_data

,tweetID,tweetText,docno,text
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",549679192804061184,الاعدام لعامل مطعم قتل زميله طعنا البيادر أيدت محكمة التمييز الحكم الصادر محكمة الجنايات الكبرى والقاضي
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,549699343666532352,الأخبار تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع أجلت محكمة الجنايات بالعاصمة تاريخ لاحق محاكمة سبعة إ
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,549711593487888387,عشآن أعطيتك وحده صميم صرت ترمي أعذار حقق العالميةة أرجع كلمني ي الأياب الانتحاري
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,549719610459967488,النهدي ثمانية قتلى تفجير انتحاري بسيارة مفخخة معملين للغاز ريف حمص شبكة الصين
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,549720880717508608,البحرين ضبط مطلوبين متورطين التفجير بالعكر الشرقي بقية الموضوع اضغط
...,...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,561985373048299520,مواسيا الشعب السعودي حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة الملك عبدالله الخبر السعودية
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,561987332878766081,تم تفجير صماخنا
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,561988825186971650,جزاك الله ألف خير جزاك الله خير كلمة ألف تحجير لخير الله
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,561991173360091136,نفذت النصرة عملية تفجير الحافلة اللبنانية دمشق


load queries (topics titles) that are already defined and released with EveTAR dataset

In [ ]:
#read the topics file from Github and use the titles as queries
topics=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/topics.txt", sep='\t',names=['data'])
queries=[]
qid=[]
#we will get the queries and their ids from the topics file
for i in range(len(topics)):
    splitted=topics["data"][i:i+1][i].split(" ")
    if splitted[0]=="<title>":
       queries.append(' '.join(splitted[1:]))
    if splitted[0]=="<num>":
       qid.append(splitted[2])

queriesDF=pd.DataFrame()
queriesDF["qid"]=qid
queriesDF["query"]=queries

queriesDF

,qid,query
0,E01,تفجير انتحاري في اب
1,E02,ليتوانيا تستخدم اليورو بدل الليتاس
2,E03,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية
3,E04,وفاة أبو أنس الليبي في نيويورك
4,E05,اختراق كوريا الشمالية حسابات سوني
5,E06,بناء أول كنيسة في إسطنبول منذ قرن
6,E07,باباندريو يؤسس حزب جديد
7,E08,بوكو حرام تخطف شباب في نيجريا
8,E09,سيطرة بوكو حرام على قاعدة عسكرية في نيجيريا
9,E10,مقتل مسلحين في غارات في باكستان


In [ ]:
# apply the same prepration steps on the queries
queriesDF["query"]=queriesDF["query"].apply(preprocess2)
queriesDF

,qid,query
0,E01,تفجير انتحاري اب
1,E02,ليتوانيا تستخدم اليورو بدل الليتاس
2,E03,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية
3,E04,وفاة أنس الليبي نيويورك
4,E05,اختراق كوريا الشمالية حسابات سوني
5,E06,بناء كنيسة إسطنبول قرن
6,E07,باباندريو يؤسس حزب جديد
7,E08,بوكو حرام تخطف شباب نيجريا
8,E09,سيطرة بوكو حرام قاعدة عسكرية نيجيريا
9,E10,مقتل مسلحين غارات باكستان


### Create index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
indexer = pt.DFIndexer("/content/drive/MyDrive/myFirstIndex", overwrite=True)
indexer.setProperty("tokeniser", "UTFTokeniser")

index_ref = indexer.index(full_data['text'], full_data["docno"])
index_ref.toString()

/tmp/ipython-input-605290042.py:1: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("/content/drive/MyDrive/myFirstIndex", overwrite=True)


'/content/drive/MyDrive/myFirstIndex/data.properties'

In [ ]:


index = pt.IndexFactory.of(index_ref)

In [ ]:
# Load index
# we can use this command to load the index after creation on our drive
#index_ref = pt.IndexRef.of("/content/drive/MyDrive/myFirstIndex/data.properties")
#index = pt.IndexFactory.of(index_ref)


### Retrieval Models


In [ ]:


tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=1000)

bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=1000)




/tmp/ipython-input-1259110819.py:3: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=1000)
/tmp/ipython-input-1259110819.py:7: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=1000)


### Testing


In [ ]:
tfidf_res=tfidf_retr.transform(queriesDF)

save_path = "/content/drive/MyDrive/tfidf.txt"
tfidf_res.to_csv(save_path,encoding="utf-8", sep="\t", index=False)

tfidf_res[:10]

,qid,docid,docno,rank,score,query
0,E01,478,550280160944328705,0,12.337848,تفجير انتحاري اب
1,E01,224,550214847833862146,1,11.339979,تفجير انتحاري اب
2,E01,1685,550686145386799107,2,10.491444,تفجير انتحاري اب
3,E01,431,550270455861751808,3,10.113079,تفجير انتحاري اب
4,E01,389,550253489973366784,4,9.761055,تفجير انتحاري اب
5,E01,407,550258502565122050,5,9.761055,تفجير انتحاري اب
6,E01,360,550244798163333120,6,8.074644,تفجير انتحاري اب
7,E01,401,550256794128613376,7,8.074644,تفجير انتحاري اب
8,E01,22399,554039453467942912,8,7.490294,تفجير انتحاري اب
9,E01,903,550407131271548928,9,7.358679,تفجير انتحاري اب


In [ ]:

bm25_res=bm25_retr.transform(queriesDF)

save_path = "/content/drive/MyDrive/Bm25.txt"
bm25_res.to_csv(save_path,encoding="utf-8", sep="\t", index=False)


bm25_res[:10]



,qid,docid,docno,rank,score,query
0,E01,478,550280160944328705,0,22.358271,تفجير انتحاري اب
1,E01,224,550214847833862146,1,20.549962,تفجير انتحاري اب
2,E01,1685,550686145386799107,2,19.012274,تفجير انتحاري اب
3,E01,431,550270455861751808,3,18.326613,تفجير انتحاري اب
4,E01,389,550253489973366784,4,17.688686,تفجير انتحاري اب
5,E01,407,550258502565122050,5,17.688686,تفجير انتحاري اب
6,E01,360,550244798163333120,6,14.632623,تفجير انتحاري اب
7,E01,401,550256794128613376,7,14.632623,تفجير انتحاري اب
8,E01,22399,554039453467942912,8,13.449196,تفجير انتحاري اب
9,E01,903,550407131271548928,9,13.435284,تفجير انتحاري اب


### **Evaluating our results**
To evaluate the results we need qrels (relevance judgements). The qrels should be in [TREC format](https://trec.nist.gov/).

In [ ]:

qrels=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/qrels.txt", sep='\t',names=['qid','Q0','docno','label'])
qrels['docno']=qrels['docno'].astype(str)
qrels['qid']=qrels['qid'].astype(str)


pt.Experiment(
[bm25_res, tfidf_res],
queriesDF,
qrels,
eval_metrics=["map","recall","P"],
names=["bm25_retr","tfidf_retr"]
)

,name,map,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000
0,bm25_retr,0.443920,0.015761,0.030743,0.045563,0.059874,0.083196,0.209260,0.331789,0.512458,0.613898,0.86,0.854,0.844,0.834,0.816667,0.7152,0.6170,0.43968,0.28278
1,tfidf_retr,0.442752,0.015761,0.030855,0.045563,0.059874,0.083241,0.208986,0.332160,0.512183,0.612491,0.86,0.856,0.844,0.834,0.818000,0.7142,0.6168,0.43832,0.28212


In [ ]:
BM25=pd.read_csv("/content/drive/MyDrive/tfidf.txt", sep='\t', header=0)
BM25['docno']=BM25['docno'].astype(str)
BM25['qid']=BM25['qid'].astype(str)


TFIDF=pd.read_csv("/content/drive/MyDrive/Bm25.txt", sep='\t', header=0)
TFIDF['docno']=TFIDF['docno'].astype(str)
TFIDF['qid']=TFIDF['qid'].astype(str)


pt.Experiment(
[BM25, TFIDF],
queriesDF,
qrels,
eval_metrics=["map","recall","P"],
names=["bm25_retr","tfidf_retr"]
)

,name,map,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000
0,bm25_retr,0.442752,0.015761,0.030855,0.045563,0.059874,0.083241,0.208986,0.332160,0.512183,0.612491,0.86,0.856,0.844,0.834,0.818000,0.7142,0.6168,0.43832,0.28212
1,tfidf_retr,0.443920,0.015761,0.030743,0.045563,0.059874,0.083196,0.209260,0.331789,0.512458,0.613898,0.86,0.854,0.844,0.834,0.816667,0.7152,0.6170,0.43968,0.28278


### **References**


* [PyTerrier  retrieval and evaluation notebook](https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/retrieval_and_evaluation.ipynb).
*   [PyTerrier documentation.](https://pyterrier.readthedocs.io/_/downloads/en/latest/pdf/)

* Tamer Elsayed, Lectures notes and Labs, Qatar University  
